# Fine-Tuning with Hugging Face PEFT
**Goal**: This tutorial demonstrates how to fine-tune a pretrained transformer model using **Parameter-Efficient Fine-Tuning (PEFT)** with LoRA on a text classification task. The goal is to reduce computational cost while maintaining effectiveness.

## 1. Introduction to PEFT
**Parameter-Efficient Fine-Tuning (PEFT)** is a suite of techniques that enables training only a small number of parameters.

Common techniques include:
- **LoRA**: Low-Rank Adaptation layers added to existing weights.
- **Prefix Tuning**: Adds trainable vectors to transformer input sequences.
- **Prompt Tuning**: Learns soft embeddings to prompt the model.

### Why PEFT?
- Reduces memory and compute requirements.
- Supports multi-task fine-tuning without full model duplication.
- Great for edge devices and low-budget environments.

### Mathematical View: LoRA
Let $W \in \mathbb{R}^{d \times k}$ be a frozen pretrained weight matrix. We introduce trainable matrices $A \in \mathbb{R}^{d \times r}$ and $B \in \mathbb{R}^{r \times k}$. The update becomes:

$W' = W + AB$

## 2. Model Types and Selection
- **Encoder-Only**: e.g., BERT, RoBERTa, DistilBERT – best for classification/token-level tasks
- **Decoder-Only**: e.g., GPT-2, GPT-J, GPT-Neo – great for generative tasks
- **Encoder-Decoder**: e.g., T5, BART, mT5 – good for seq2seq tasks like summarization or translation

In this project, we use **encoder-only BERT** to perform **text classification** using LoRA from the PEFT library.

## 3. Environment Setup
Install the required packages using pip:

In [1]:
!pip install transformers peft datasets accelerate scikit-learn torch

## 4. Load and Prepare Dataset

In [2]:
from datasets import Dataset

samples = [
    {"text": "I love this product!", "label": 1},
    {"text": "Horrible experience.", "label": 0},
    {"text": "Fantastic!", "label": 1},
    {"text": "I want a refund.", "label": 0},
]

dataset = Dataset.from_list(samples)
dataset = dataset.train_test_split(test_size=0.25)

/home/user/anaconda3/envs/henry3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 5. Tokenization and Model Loading

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

def preprocess(example):
    return tokenizer(example['text'], truncation=True, padding="max_length", max_length=64)

dataset = dataset.map(preprocess, batched=True)
dataset = dataset.remove_columns(["text"])
dataset.set_format("torch")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map: 100%|██████████| 1/1 [00:00<00:00, 261.64 examples/s]


## 6. Apply LoRA with PEFT

In [4]:
from peft import get_peft_model, LoraConfig, TaskType

config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
)

model = get_peft_model(model, config)
model.print_trainable_parameters()

trainable params: 296,450 || all params: 109,780,228 || trainable%: 0.2700395193203643


## 7. Training the Model

In [6]:
from transformers import TrainingArguments, Trainer

# Define training arguments with W&B logging disabled properly
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    evaluation_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    report_to="none",
)

# Create Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
)

# Train
trainer.train()


/home/user/anaconda3/envs/henry3/lib/python3.12/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False)
  warnings.warn(
  0%|          | 0/3 [00:00<?, ?it/s]/home/user/anaconda3/envs/henry3/lib/python3.12/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
                                             
 67%|██████▋   | 2/3 [00:03<00:01,  1.62s/it]

{'eval_loss': 0.6087948083877563, 'eval_runtime': 0.1352, 'eval_samples_per_second': 7.395, 'eval_steps_per_second': 7.395, 'epoch': 1.0}


                                             
100%|██████████| 3/3 [00:04<00:00,  1.02it/s]

{'eval_loss': 0.618904173374176, 'eval_runtime': 0.0535, 'eval_samples_per_second': 18.699, 'eval_steps_per_second': 18.699, 'epoch': 2.0}


                                             
100%|██████████| 3/3 [00:04<00:00,  1.36s/it]

{'eval_loss': 0.6238903403282166, 'eval_runtime': 0.0543, 'eval_samples_per_second': 18.416, 'eval_steps_per_second': 18.416, 'epoch': 3.0}
{'train_runtime': 4.0898, 'train_samples_per_second': 2.201, 'train_steps_per_second': 0.734, 'train_loss': 0.7400258382161459, 'epoch': 3.0}


TrainOutput(global_step=3, training_loss=0.7400258382161459, metrics={'train_runtime': 4.0898, 'train_samples_per_second': 2.201, 'train_steps_per_second': 0.734, 'train_loss': 0.7400258382161459, 'epoch': 3.0})

## 8. Inference

In [7]:
from transformers import pipeline

# Create an inference pipeline using the trained model and tokenizer
clf_pipeline = pipeline("text-classification", model=model, tokenizer=tokenizer, return_all_scores=False)

# Raw test examples (re-adding original text manually since it was removed from dataset)
test_texts = ["Fantastic!", "I want a refund."]

# Run inference
for text in test_texts:
    prediction = clf_pipeline(text)[0]
    print(f"Input: {text}")
    print(f"Predicted label: {prediction['label']} with score {prediction['score']:.4f}\n")


/home/user/anaconda3/envs/henry3/lib/python3.12/site-packages/transformers/pipelines/text_classification.py:105: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(
The model 'PeftModelForSequenceClassification' is not supported for text-classification. Supported models are ['AlbertForSequenceClassification', 'BartForSequenceClassification', 'BertForSequenceClassification', 'BigBirdForSequenceClassification', 'BigBirdPegasusForSequenceClassification', 'BioGptForSequenceClassification', 'BloomForSequenceClassification', 'CamembertForSequenceClassification', 'CanineForSequenceClassification', 'LlamaForSequenceClassification', 'ConvBertForSequenceClassification', 'CTRLForSequenceClassification', 'Data2VecTextForSequenceClassification', 'DebertaForSequenceClassification', 'DebertaV2ForSequenceClassification', 'DistilBertForSequenceClassif

Input: Fantastic!
Predicted label: LABEL_1 with score 0.5084

Input: I want a refund.
Predicted label: LABEL_1 with score 0.5367



## 9. Summary and Use Cases
LoRA fine-tuning allows fast adaptation of large models in resource-constrained environments.

**Use cases:**
- Fine-tuning foundation models with small datasets
- Efficient multi-task training
- On-device NLP applications